**Use of Grid Search with CatBoost Regressor on the Boston Housing data**

**AWS Configuration required**

Fit2ec2 use your credentials stored in ~/.aws/credential

Make sure you have install aws-cli and it's configured

https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
# Import the usual suspects
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import pandas as pd
# fit2ec2
import fit2ec2 as fe

<IPython.core.display.Javascript object>

**Load and Split data**

In [ ]:
data = pd.read_csv('data.csv')

y = np.log(data.SalePrice)
X = data.drop(['SalePrice', 'Id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
                          X, y, random_state=42, test_size=.33)

print(X_train.shape)

**Initialize the Random Forest and the Grid Search**

In [ ]:
# Don't forget the n_jobs parameters on a multi-cpu machine!
param_grid = {'learning_rate': np.arange(0.05, 0.5, 0.01),
        'depth': range(1,3,1),
        'l2_leaf_reg': np.arange(0.1,2.0,0.1),
        #'random_seed':42,
        #'ignored_features':['Id','SalePrice'],
        #'one_hot_max_size':10
             }

model = CatBoostRegressor()
grid = model.grid_search(param_grid,
            X_train,
            y=y_train,
            cv=5,
            #partition_random_seed=0,
            #calc_cv_statistics=True,
            #search_by_train_test_split=False,
            #refit=True,
            #shuffle=True,
            #stratified=None,
            #train_size=0.8,
            #verbose=True,
            plot=True)

**Initialize Fit2ec2**

In [ ]:
cmp = fe.Compute()

# Or specify the private key name
# cmpC = fe.Fit2ec2(keyname="mykey")

**Create the ec2 instance**

In [ ]:
# Create an EC2 with the default AWS Linux AMI and a type of t2.micro
cmp.create()#instanceType="c5.12xlarge")

# Or specify the image id (AMI ID) and the instance type (see https://aws.amazon.com/ec2/instance-types/)
#cmp.create(imageId='ami-06ce3edf0cff21f07', instanceType='t2.micro')

**Fit the model with the data on the ec2 instance**

In [ ]:
grid = cmp.fit(grid, X_train, y_train)
# If you get NoValidConnectionsError, better to wait 30secs for the ec2 instance to turn on

In [ ]:
grid.best_score_

In [ ]:
grid.best_estimator_

**Terminate the ec2 instance and remove the private key**

In [ ]:
cmp.terminate()